In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import os
from scripts import text_utils
%load_ext autoreload
%autoreload 2

In [2]:
raw_text = text_utils.get_two_cities_sample(5000)

In [3]:
text = text_utils.clean_up_text(raw_text)

In [4]:
text[:200]

'the period \n it was the best of times , it was the worst of times , it was the age of wisdom , it was the age of foolishness , it was the epoch of belief , it was the epoch of incredulity , it was the'

### Convert text corpus to integers

In [5]:
text.split(' ')[:10]

['the', 'period', '\n', 'it', 'was', 'the', 'best', 'of', 'times', ',']

In [6]:
word_dict = text_utils.WordDict(text.split(' '))
print(word_dict.get_id('light'))
print(word_dict.get_id('the2'))
print(word_dict.get_token(200))
print(word_dict.tokens_to_ids(text.split(' ')[:100]))
print(word_dict.ids_to_tokens([2,3,5,6,4,2,6]))

18
0
spending
[1, 2, 3, 4, 5, 1, 6, 7, 8, 9, 4, 5, 1, 10, 7, 8, 9, 4, 5, 1, 11, 7, 12, 9, 4, 5, 1, 11, 7, 13, 9, 4, 5, 1, 14, 7, 15, 9, 4, 5, 1, 14, 7, 16, 9, 4, 5, 1, 17, 7, 18, 9, 4, 5, 1, 17, 7, 19, 9, 4, 5, 1, 20, 7, 21, 9, 4, 5, 1, 22, 7, 23, 9, 24, 25, 26, 27, 28, 9, 24, 25, 29, 27, 28, 9, 24, 30, 31, 32, 33, 34, 35, 9, 24, 30, 31, 32, 33, 1, 36]
['period', '\n', 'was', 'best', 'it', 'period', 'best']


In [7]:
print(word_dict.get_vocab_size())

5557


### Word2Vec Word Embedding

In [8]:
vocabulary_size = word_dict.get_vocab_size() # TODO
embedding_size = 300
batch_size = 64
paragraph_length = 30
hidden_dim = 40

lstm_size = 100

In [14]:
# # hyperparameters
# num_sampled = 3

# # build the graph
# g = tf.Graph()
# with g.as_default():
#   embeddings = tf.Variable(
#     tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0),
#     name="embeddings")
#   words = tf.placeholder(tf.int32, shape=[batch_size, paragraph_length])
  
#   # initial hidden state
#   h0 = tf.placeholder(tf.float32, shape=[hidden_dim])
#   # inputs to cells
#   X = tf.nn.embedding_lookup(embeddings, words)

#   cell = tf.contrib.rnn.BasicLSTMCell(num_cells)
  
  
#   cell(hp)
  

In [17]:
g = tf.Graph()
with g.as_default():
  cell = tf.contrib.rnn.LSTMCell(200)
  print(cell.state_size.h)
  # Initial state of the LSTM memory.
#   state = tf.zeros([batch_size, lstm.state_size])

200


In [18]:

# prepare feed data for training
data_full = np.array(word_dict.tokens_to_ids(text.split(' ')))
data_input = data_full[0:-1]
data_label = data_full[1:]

print(data_input.shape)
print(data_label.shape)

def gen_batch(batch_size):
  p = np.random.choice(range(len(data_input)), size=batch_size)
  return {
    train_inputs: data_input[p],
    train_labels: data_label[p, np.newaxis]
  }
  

(53740,)
(53740,)


In [19]:
LOG_DIR = './visual/embed/'

In [20]:
# write vocab to metadata.tsv
with open(os.path.join(LOG_DIR, 'metadata.tsv'), 'w') as f:
  for i in range(word_dict.get_vocab_size()):
    if word_dict.get_token(i) == '\n':
      f.write('\\n' + '\n')
    else:
      f.write(word_dict.get_token(i) + '\n')

In [21]:
from tensorflow.contrib.tensorboard.plugins import projector

config = projector.ProjectorConfig()
embedding_config = config.embeddings.add()
embedding_config.tensor_name = embeddings.name
embedding_config.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')


In [26]:
with tf.Session(graph=g) as sess:
  graph_writer = tf.summary.FileWriter(LOG_DIR)
  graph_writer.add_graph(sess.graph)
  
  projector.visualize_embeddings(graph_writer, config)
  sess.run(tf.global_variables_initializer())
  
  saver = tf.train.Saver()
  
  batch_size = 32
  for i in range(6000):
    _, cur_loss = sess.run([train_op, loss], feed_dict=gen_batch(batch_size))
    if i % 100 == 0:
      print("iter: {}, training loss: {}".format(i, cur_loss))
    
  saver.save(sess, os.path.join(LOG_DIR, 'embed_model.ckpt'), global_step=i)

iter: 0, training loss: 19.49799919128418
iter: 100, training loss: 12.179189682006836
iter: 200, training loss: 18.494905471801758
iter: 300, training loss: 19.68937110900879
iter: 400, training loss: 22.087669372558594
iter: 500, training loss: 19.459903717041016
iter: 600, training loss: 11.850862503051758
iter: 700, training loss: 16.97174835205078
iter: 800, training loss: 17.621646881103516
iter: 900, training loss: 19.054561614990234
iter: 1000, training loss: 14.592487335205078
iter: 1100, training loss: 14.503328323364258
iter: 1200, training loss: 18.040023803710938
iter: 1300, training loss: 11.424406051635742
iter: 1400, training loss: 12.496201515197754
iter: 1500, training loss: 5.899695873260498
iter: 1600, training loss: 14.117030143737793
iter: 1700, training loss: 14.185291290283203
iter: 1800, training loss: 9.79128360748291
iter: 1900, training loss: 11.126241683959961
iter: 2000, training loss: 17.155893325805664
iter: 2100, training loss: 11.354045867919922
iter: 